In [ ]:

import datetime
import sys

#Define variables   
start_time = datetime.datetime.now();
########---↓↓↓---############添加源表和目标表
tgt_objectname = 'dwd_ms.busi_ln_sales_transaction_data';
src_objectname = 'prep_ms_row.ecc_vbkd';
########---↑↑↑---############添加源表和目标表
version_str = spark.sql(f"""
                             select max(data_watermark_value) as version from dwd_ms.edw_loading_control where tgt_objectname = '{tgt_objectname}' and src_objectname = '{src_objectname}'
                         """).collect()[0]['version'];

last_commit_time = datetime.datetime.now();

#Get version variables
if version_str == None:
    version_str = 0
else:
    #Get last end_time in edw_loading_control list
    last_commit_time = spark.sql(f"""
                                      select commit_time from dwd_ms.edw_loading_control where tgt_objectname = '{tgt_objectname}' and src_objectname = '{src_objectname}'
                                  """).collect()[0]['commit_time']
    
    #Get this commit_timestamp in change_log list 
    this_commit_time = spark.sql(f"""
                                      select _commit_timestamp as commit_timestamp from table_changes('{src_objectname}',0)
                                      where _commit_version <= int({version_str})
                                  """).collect()[0]['commit_timestamp']

    new_version = spark.sql(f"""
                                 select min(_commit_version) as new_version from table_changes('{src_objectname}',0)
                                 where _commit_version > int({version_str})
                                   and _change_type <> 'update_preimage'
                                 ;
                             """).collect()[0]['new_version'];

    print('last_commit_time = ' + str(last_commit_time));
    print('this_commit_time = ' + str(this_commit_time));
            
version = 0;

if version_str == 0:
    print('No data in the edw_loading_control list!')
    version = 0
else:
    print('Have the data in the edw_loading_control list!')
    #Determines whether the source table initializes data
    if this_commit_time > last_commit_time:
        print('The source table was drop, full capacity run number')
        version = 0
    else:
        if new_version == None:
            print('No new data')
            sys.exit()
        else:
            version = int(new_version);
            
            
max_version = int(spark.sql(f"""
                                 select max(_commit_version) as max_version from table_changes('{src_objectname}',{version})
                                 ;
                             """).collect()[0]['max_version']);

print('version = %s' %version);
print('max_version = %s' %max_version);

########---↓↓↓---#########添加源表和目标表sql语句
#Logic processing section
run_full = f"""
           create or replace temp view tmp_ecc_vbkd_full  
           as
            select 
                 vbeln 	                    as sales_transaction_no
                ,posnr 	                    as sales_transaction_ln_no
                ,inco1 	                    as incoterm1_cd
                ,inco2 	                    as incoterm2_cd
                ,bstkd 	                    as cust_po_no
                ,posex_e                    as cust_po_ln_no
                ,bstdk 	                    as cust_po_dt
                ,zlsch 	                    as prepayment_mthd_ind
                ,ihrez 	                    as your_reference_desc
                ,ihrez_e                    as ship_to_reference_desc
                ,kdkg2 	                    as cust_condition_group_desc
                ,lcnum 	                    as fin_doc_no
                ,akprz 	                    as depreciation_percent
                ,zterm 	                    as pymt_terms_cd
                ,abssc 	                    as pymt_guarantee_cd
                ,bsark 	                    as cust_po_type
                ,rrrel	                    as rev_recognition_catg_cd
                ,nvl(__dl_source_id,'') 	as dw_source_id
                ,now()                      as dw_insert_dt
                ,now()                      as dw_update_dt
                ,''                         as dw_del_flg
                ,case when  __src_record_del_status = 'D' then 'delete'
                 else ''      end    		as change_type
            from prep_ms_row.ecc_vbkd
           where mandt = '301'
                 ;
             """;

run_increment = f"""
     create or replace temp view tmp_ecc_vbkd_incr 
     as    
     select
           sales_transaction_no
          ,sales_transaction_ln_no
          ,incoterm1_cd
          ,incoterm2_cd
          ,cust_po_no
          ,cust_po_ln_no
          ,cust_po_dt
          ,prepayment_mthd_ind
          ,your_reference_desc
          ,ship_to_reference_desc
          ,cust_condition_group_desc
          ,fin_doc_no
          ,depreciation_percent
          ,pymt_terms_cd
          ,pymt_guarantee_cd
          ,cust_po_type             --Customer PO TYPE
          ,rev_recognition_catg_cd --revenue recognition category code
          ,dw_source_id
          ,dw_insert_dt
          ,dw_update_dt
          ,dw_del_flg 
          ,change_type
     from (select 
                 a.vbeln 	                    as sales_transaction_no
                ,a.posnr 	                    as sales_transaction_ln_no
                ,a.inco1 	                    as incoterm1_cd
                ,a.inco2 	                    as incoterm2_cd
                ,a.bstkd 	                    as cust_po_no
                ,a.posex_e                      as cust_po_ln_no
                ,a.bstdk 	                    as cust_po_dt
                ,a.zlsch 	                    as prepayment_mthd_ind
                ,a.ihrez 	                    as your_reference_desc
                ,a.ihrez_e                      as ship_to_reference_desc
                ,a.kdkg2 	                    as cust_condition_group_desc
                ,a.lcnum 	                    as fin_doc_no
                ,a.akprz 	                    as depreciation_percent
                ,a.zterm 	                    as pymt_terms_cd
                ,a.abssc 	                    as pymt_guarantee_cd
                ,a.bsark 	                    as cust_po_type
                ,a.rrrel	                    as rev_recognition_catg_cd
                ,nvl(a.__dl_source_id,'') 	    as dw_source_id
                ,now()                          as dw_insert_dt
                ,now()                          as dw_update_dt
                ,''                             as dw_del_flg
                ,case when  a.__src_record_del_status = 'D' then 'delete'
                 else a.change_type    end  as change_type
                ,row_number over(partition by a.vbeln ,a.posnr order by a._commit_timestamp desc ) as rn 
           from table_changes({src_objectname},{version},{max_version})  a 
          where a.mandt = '301'
            and a._commit_timestamp >= cast('{last_commit_time}' as timestamp)
            and a._change_type <> 'update_preimage'
           ) t
     where rn = 1 ;
              """;
              
if version == 0:
    print('For the first time to run or the source table was dropped!')
    spark.sql(run_full);
    source_data = 'tmp_ecc_vbkd_full'

else:
    print('More than two runs!')
    spark.sql(run_increment)
    source_data = 'tmp_ecc_vbkd_incr'

common_run1 = f"""
               create or replace temp view except_busi_ln_sales_transaction_data_v as
                select  *,
                        ''   as  change_type
                from 
                   (
                  select
                        sales_transaction_no
                       ,sales_transaction_ln_no
                       ,incoterm1_cd
                       ,incoterm2_cd
                       ,cust_po_no
                       ,cust_po_ln_no
                       ,cust_po_dt
                       ,prepayment_mthd_ind
                       ,your_reference_desc
                       ,ship_to_reference_desc
                       ,cust_condition_group_desc
                       ,fin_doc_no
                       ,depreciation_percent
                       ,pymt_terms_cd
                       ,pymt_guarantee_cd
                       ,cust_po_type             --Customer PO TYPE
                       ,rev_recognition_catg_cd  --revenue recognition category code
                       ,dw_source_id
                       ,dw_insert_dt
                       ,dw_update_dt
                       ,dw_del_flg
                  from {source_data}  
                 where change_type <> 'delete'
                except 
                select
                        sales_transaction_no
                       ,sales_transaction_ln_no
                       ,incoterm1_cd
                       ,incoterm2_cd
                       ,cust_po_no
                       ,cust_po_ln_no
                       ,cust_po_dt
                       ,prepayment_mthd_ind
                       ,your_reference_desc
                       ,ship_to_reference_desc
                       ,cust_condition_group_desc
                       ,fin_doc_no
                       ,depreciation_percent
                       ,pymt_terms_cd
                       ,pymt_guarantee_cd
                       ,cust_po_type             --Customer PO TYPE
                       ,rev_recognition_catg_cd  --revenue recognition category code
                       ,dw_source_id
                       ,dw_insert_dt
                       ,dw_update_dt
                       ,dw_del_flg
                  from dwd_ms.busi_ln_sales_transaction_data
                   )  t
                union
               select
                     sales_transaction_no
                    ,sales_transaction_ln_no
                    ,incoterm1_cd
                    ,incoterm2_cd
                    ,cust_po_no
                    ,cust_po_ln_no
                    ,cust_po_dt
                    ,prepayment_mthd_ind
                    ,your_reference_desc
                    ,ship_to_reference_desc
                    ,cust_condition_group_desc
                    ,fin_doc_no
                    ,depreciation_percent
                    ,pymt_terms_cd
                    ,pymt_guarantee_cd
                    ,cust_po_type             --Customer PO TYPE
                    ,rev_recognition_catg_cd  --revenue recognition category code
                    ,dw_source_id
                    ,dw_insert_dt
                    ,dw_update_dt
                    ,dw_del_flg
                    ,change_type 
               from {source_data}  
               where change_type = 'delete'  
            """
common_run2 = f"""      
                merge into dwd_ms.busi_ln_sales_transaction_data    t
                using except_busi_ln_sales_transaction_data_v  v
                   on t.sales_transaction_no = v.sales_transaction_no
                  and t.sales_transaction_ln_no = v.sales_transaction_ln_no 
                 when matched and v.change_type = 'delete'  then delete 
                 when matched and v.change_type <> 'delete' then
               update set
                      t.sales_transaction_no      = v.sales_transaction_no                                            
                     ,t.sales_transaction_ln_no   = v.sales_transaction_ln_no                              
                     ,t.incoterm1_cd              = v.incoterm1_cd                                                      
                     ,t.incoterm2_cd              = v.incoterm2_cd                                                    
                     ,t.cust_po_no                = v.cust_po_no                                           
                     ,t.cust_po_ln_no             = v.cust_po_ln_no                                   
                     ,t.cust_po_dt                = v.cust_po_dt                                            
                     ,t.prepayment_mthd_ind       = v.prepayment_mthd_ind                                        
                     ,t.your_reference_desc       = v.your_reference_desc                                                  
                     ,t.ship_to_reference_desc    = v.ship_to_reference_desc                                 
                     ,t.cust_condition_group_desc = v.cust_condition_group_desc                                                 
                     ,t.fin_doc_no                = v.fin_doc_no                                    
                     ,t.depreciation_percent      = v.depreciation_percent                                          
                     ,t.pymt_terms_cd             = v.pymt_terms_cd                                    
                     ,t.pymt_guarantee_cd         = v.pymt_guarantee_cd                                              
                     ,t.cust_po_type              = v.cust_po_type                                                        
                     ,t.rev_recognition_catg_cd   = v.rev_recognition_catg_cd                           
                     ,t.dw_source_id              = v.dw_source_id                             
                     ,t.dw_insert_dt              = v.dw_insert_dt                                            
                     ,t.dw_update_dt              = v.dw_update_dt                                                     
                     ,t.dw_del_flg                = v.dw_del_flg                                                  
                when not matched and v.change_type <> 'delete' then  
              insert 
                    (
                     t.sales_transaction_no     
                    ,t.sales_transaction_ln_no  
                    ,t.incoterm1_cd             
                    ,t.incoterm2_cd             
                    ,t.cust_po_no               
                    ,t.cust_po_ln_no            
                    ,t.cust_po_dt               
                    ,t.prepayment_mthd_ind      
                    ,t.your_reference_desc      
                    ,t.ship_to_reference_desc   
                    ,t.cust_condition_group_desc
                    ,t.fin_doc_no               
                    ,t.depreciation_percent     
                    ,t.pymt_terms_cd            
                    ,t.pymt_guarantee_cd        
                    ,t.cust_po_type             
                    ,t.rev_recognition_catg_cd  
                    ,t.dw_source_id             
                    ,t.dw_insert_dt             
                    ,t.dw_update_dt             
                    ,t.dw_del_flg    
                    )
                    values 
                    (
                     v.sales_transaction_no     
                    ,v.sales_transaction_ln_no  
                    ,v.incoterm1_cd             
                    ,v.incoterm2_cd             
                    ,v.cust_po_no               
                    ,v.cust_po_ln_no            
                    ,v.cust_po_dt               
                    ,v.prepayment_mthd_ind      
                    ,v.your_reference_desc      
                    ,v.ship_to_reference_desc   
                    ,v.cust_condition_group_desc
                    ,v.fin_doc_no               
                    ,v.depreciation_percent     
                    ,v.pymt_terms_cd            
                    ,v.pymt_guarantee_cd        
                    ,v.cust_po_type             
                    ,v.rev_recognition_catg_cd  
                    ,v.dw_source_id             
                    ,v.dw_insert_dt             
                    ,v.dw_update_dt             
                    ,v.dw_del_flg      
                    )
                """
spark.sql(common_run1)
spark.sql(common_run2)

end_time = datetime.datetime.now();

max_commit_time = spark.sql(f"""
                                 select max(_commit_timestamp) as commit_time from table_changes('{src_objectname}',{version},{max_version})
                             """
                             ).collect()[0]['commit_time'];

spark.sql(f"""
           create or replace temp view edw_loading_control_v as           
            select  '{tgt_objectname}' as tgt_objectname
                   ,'{src_objectname}' as src_objectname
                   ,'{start_time}' as start_time
                   ,'{end_time}' as end_time 
                   ,'{max_version}' as data_watermark_value 
                   ,'{max_commit_time}' as commit_time  
              """ )
spark.sql(f"""
         merge into dwd_ms.edw_loading_control     tab1
         using edw_loading_control_v      tab2
            on tab1.tgt_objectname = tab2.tgt_objectname 
           and tab1.src_objectname = tab2.src_objectname 
          when matched then 
        update set 
               tab1.start_time = tab2.start_time ,
               tab1.end_time   = tab2.end_time ,
               tab1.data_watermark_value = tab2.data_watermark_value ,
               tab1.commit_time = tab2.commit_time  
          when not matched then 
        insert 
              (
               tab1.tgt_objectname
              ,tab1.src_objectname
              ,tab1.start_time
              ,tab1.end_time
              ,tab1.data_watermark_type
              ,tab1.data_watermark_value
              ,tab1.process_status
              ,tab1.error_number
              ,tab1.error_message
              ,tab1.commit_time
              )
        values
             (
               tab2.tgt_objectname
              ,tab2.src_objectname
              ,tab2.start_time
              ,tab2.end_time
              ,null
              ,tab2.data_watermark_value
              ,null
              ,null
              ,null
              ,tab2.commit_time
              )  """)
########---↑↑↑---#########添加源表和目标表sql语句